# RED NEURONAL CONVOLUCIONAL - PREDICCIÓN DE EXPRESIONES FACIALES

## Acerca del conjunto de datos
Los datos consisten en imágenes de rostros en escala de grises de 48 x 48 píxeles. Los rostros se han registrado automáticamente de modo que estén más o menos centrados y ocupen aproximadamente la misma cantidad de espacio en cada imagen.

La tarea consiste en categorizar cada rostro en función de la emoción que se muestra en la expresión facial en una de siete categorías (0 = Enfado, 1 = Disgusto, 2 = Miedo, 3 = Felicidad, 4 = Tristeza, 5 = Sorpresa, 6 = Neutral). El conjunto de entrenamiento consta de 28.709 ejemplos y el conjunto de prueba público consta de 3.589 ejemplos.

## Importación de datos

### Importación de la biblioteca de descarga del dataset de expresiones faciales

In [ ]:
!pip install kagglehub

### Importación del dataset al entorno local

In [ ]:
import os
import shutil
import kagglehub

# shutil.rmtree("/root/.cache/kagglehub", ignore_errors=True)
# shutil.rmtree("./dataset", ignore_errors=True)

# Ruta local del dataset organizado
dataset_local_path = "./dataset"

# Verificar si el dataset ya existe localmente
if os.path.exists(dataset_local_path) and len(os.listdir(dataset_local_path)) > 0:
  print(f"El dataset ya existe en el entorno local: {dataset_local_path}")
else:
  print("El dataset no se encontró en el entorno local. Procediendo a descargarlo...")

  # Ruta donde se descargará el dataset de AffectNet
  dataset_donwload_path = kagglehub.dataset_download("noamsegal/affectnet-training-data")

  # Verificar si la descarga contiene las carpetas esperadas
  if os.path.exists(dataset_donwload_path):
    print(f"El dataset ha sido descargado en {dataset_donwload_path}")

    raw_download_dataset_path = os.path.join(dataset_local_path, "raw_download")

    # Mover las carpetas a un directorio organizado
    os.makedirs(raw_download_dataset_path, exist_ok=True)

    # Move the contents of the source folder
    for item in os.listdir(dataset_donwload_path):

      if item in ["contempt", "disgust", "fear"]:
        continue

      source_item = os.path.join(dataset_donwload_path, item)
      destination_item = os.path.join(raw_download_dataset_path, item)
      shutil.move(source_item, destination_item)  # Move the file

    # Eliminar el dataset descargado
    shutil.rmtree("/root/.cache/kagglehub", ignore_errors=True)

    print(f"Contenido del dataset descargado movido a la carpeta {raw_download_dataset_path}")
    print(f"Dataset temporal contenido en cache eliminado exitosamente")
  else:
    print("No se encontraron las carpetas esperadas en el dataset descargado. Revisa el proceso de descarga.")

### Ordenado del dataset

In [ ]:
import random

train_path = os.path.join(dataset_local_path, "train")
test_path = os.path.join(dataset_local_path, "test")
raw_download_path = os.path.join(dataset_local_path, "raw_download")

# Crear las carpetas de train y test
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)
os.makedirs(raw_download_path, exist_ok=True)

if os.path.exists(train_path) and os.path.exists(test_path) and len(os.listdir(train_path)) > 0 and len(os.listdir(test_path)) > 0:
  print(f"Las carpetas de train y test ya existen en el entorno local: {train_path} y {test_path}")
else:
  print("Las carpetas de train y test no se encontraron en el entorno local. Procediendo a organizar el dataset...")

  # Iterar sobre las subcarpetas en raw_download (cada emoción)
  for emotion_folder in os.listdir(raw_download_path):
    emotion_path = os.path.join(raw_download_path, emotion_folder)

    # Saltar si no es una carpeta
    if not os.path.isdir(emotion_path):
      continue

    # Crear subcarpetas en train y test
    train_emotion_path = os.path.join(train_path, emotion_folder)
    test_emotion_path = os.path.join(test_path, emotion_folder)
    os.makedirs(train_emotion_path, exist_ok=True)
    os.makedirs(test_emotion_path, exist_ok=True)

    # Listar todas las imágenes en la carpeta actual
    images = [img for img in os.listdir(emotion_path) if img.endswith(('.jpg', '.png'))]

    # Mezclar las imágenes para asegurar aleatoriedad
    random.shuffle(images)

    # Calcular división 80%-20%
    split_index = int(len(images) * 0.8)
    train_images = images[:split_index]
    test_images = images[split_index:]

    # Copiar imágenes a las carpetas correspondientes
    for img in train_images:
      shutil.copy(os.path.join(emotion_path, img), os.path.join(train_emotion_path, img))
    for img in test_images:
      shutil.copy(os.path.join(emotion_path, img), os.path.join(test_emotion_path, img))

    print(f"Procesado '{emotion_folder}': {len(train_images)} imágenes en train, {len(test_images)} imágenes en test.")

  print("Reorganización completada.")

def get_folder_size(folder_path):
  total_size = 0
  for dirpath, dirnames, filenames in os.walk(folder_path):
    for file in filenames:
      file_path = os.path.join(dirpath, file)
      # Add file size
      total_size += os.path.getsize(file_path)
  return total_size

folder_size = get_folder_size(train_path)

# Convert size to a readable format (e.g., MB)
print(f"Folder size: train folder -> {folder_size / (1024 * 1024):.2f} MB")

folder_size = get_folder_size(test_path)

# Convert size to a readable format (e.g., MB)
print(f"Folder size: test folder -> {folder_size / (1024 * 1024):.2f} MB")

### Instalación de OpenCV

In [ ]:
!pip install opencv-python seaborn

### Visualización de los datos

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2

# Function to describe image properties
def get_image_properties(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
    if image is None:
        return {"Error": "Could not load image"}

    # Calculate properties
    height, width = image.shape
    size = os.path.getsize(image_path)  # File size in bytes
    mean_intensity = image.mean()  # Mean pixel intensity
    min_intensity = image.min()  # Min pixel intensity
    max_intensity = image.max()  # Max pixel intensity

    return {
        "Dimensions": f"{width}x{height}",
        "File Size (KB)": f"{size / 1024:.2f}",
        "Mean Intensity": f"{mean_intensity:.2f}",
        "Min Intensity": min_intensity,
        "Max Intensity": max_intensity,
    }

# Verify if the training folder exists
if not os.path.exists(train_path) or len(os.listdir(train_path)) == 0:
    print("The training folder is not found. Please check the dataset structure.")
else:
    print("Visualizing dataset examples...")

    # List available classes
    classes = os.listdir(train_path)
    classes.sort()  # Ensure classes are in alphabetical order
    print(f"Classes found: {classes}")

    print("Visualizing examples from each class with properties:")
    # Display an example from each class with properties
    fig, axes = plt.subplots(1, len(classes), figsize=(15, 5))
    for i, class_name in enumerate(classes):
        class_folder = os.path.join(train_path, class_name)
        if os.path.isdir(class_folder):
            # Get the first image from the class
            example_image_path = os.path.join(class_folder, os.listdir(class_folder)[0])
            image = cv2.imread(example_image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale

            # Display the image
            axes[i].imshow(image, cmap="gray")
            axes[i].axis("off")
            axes[i].set_title(class_name)

            # Get and print image properties
            properties = get_image_properties(example_image_path)
            print(f"\nClass: {class_name}")
            for prop, value in properties.items():
                print(f"{prop}: {value}")

    plt.show()

## Preparación de los datos para el entrenamiento

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # type: ignore
import tensorflow as tf

def preprocess_input_to_rgb(img):
    # Check if the image already has 3 channels
    if img.shape[-1] == 1:  # Grayscale images
        img_tensor = tf.expand_dims(img, axis=-1)  # Ensure shape is (96, 96, 1)
        return tf.image.grayscale_to_rgb(img_tensor)  # Convert to RGB (96, 96, 3)
    elif img.shape[-1] == 3:  # Already RGB
        return img  # No changes needed
    else:
        raise ValueError(f"Unexpected input shape {img.shape}")


# Image data generators
data_augmentation = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input_to_rgb,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Use 20% of data for validation
)

# Training data generator
train_generator = data_augmentation.flow_from_directory(
    directory=train_path,       # Path to the training dataset
    target_size=(96, 96),       # Resize images to 96x96
    color_mode="grayscale",           # Load as rgb images
    batch_size=32,              # Batch size
    class_mode="categorical",   # Multi-class classification
    subset="training",          # Use the "training" subset
    shuffle=True                # Shuffle data for training
)

# Validation data generator (uses the same augmentation instance with the "validation" subset)
validation_generator = data_augmentation.flow_from_directory(
    directory=train_path,
    target_size=(96, 96),
    color_mode="grayscale",
    batch_size=32,
    class_mode="categorical",
    subset="validation"       # Use the "validation" subset
)

# Define the test ImageDataGenerator with the preprocessing function
test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input_to_rgb  # Apply grayscale-to-RGB conversion
)

# Test data generator
test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=(96, 96),       # Resize test images to 96x96
    color_mode="grayscale",     # Load as grayscale images
    batch_size=32,              # Batch size
    class_mode="categorical"    # Multi-class classification
)

# Key Validation Step: Verify the shape of the generated data
x_batch, y_batch = next(test_generator)
print(f"Input batch shape: {x_batch.shape}")  # Should output (batch_size, 96, 96, 3)

# Display information about the detected classes
print(f"Classes detected: {train_generator.class_indices}")

### Verificacion del balance de clases

In [ ]:
from collections import Counter

# Obtener el mapeo de índices a nombres de clases
class_indices = train_generator.class_indices  # Diccionario {nombre_clase: índice_clase}
index_to_class = {v: k for k, v in class_indices.items()}  # Invertir el diccionario

# Contar las clases
class_counts = Counter(train_generator.classes)

# Ordenar por índice de clase y mostrar con nombres
print("Distribución de clases en entrenamiento:")
for class_index, count in sorted(class_counts.items()):
    class_name = index_to_class[class_index]  # Obtener el nombre asociado al índice
    print(f"{class_name}: {count} muestras")  

### Visualización de la distribución de pesos

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

class_counts = Counter(train_generator.classes)
plt.bar(class_counts.keys(), class_counts.values())
plt.title("Class Distribution")
plt.xlabel("Class Index")
plt.ylabel("Number of Samples")
plt.show()

### Cálculo de pesos del desbalance de clases

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Obtain class names and indices from the train generator
class_indices = train_generator.class_indices  # Class-to-index mapping
class_labels = list(class_indices.keys())  # Class names
class_indices_inverted = {v: k for k, v in class_indices.items()}  # Index-to-class mapping

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array(list(class_indices.values())),  # Use consistent class indices
    y=train_generator.classes
)

# Map weights to class labels
class_weights_dict = {class_indices_inverted[i]: weight for i, weight in enumerate(class_weights)}

# Print the computed weights
print("Class Weights:")
for class_name, weight in class_weights_dict.items():
    print(f"{class_name}: {weight:.2f}")

## Entrenamiento del modelo

### Verificación del entorno ejecutable (GPU)

In [ ]:
import tensorflow as tf
print("Dispositivos disponibles:")
print(tf.config.list_physical_devices('GPU'))

### Activación de uso de memoria dinámica de la GPU cuando sea posible

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Configuración de memoria dinámica en GPU habilitada.")
    except RuntimeError as e:
        print(e)


### Carpeta para el guardado del progreso del entrenamiento

In [ ]:
os.makedirs("model_checkpoints", exist_ok=True)

### Creación de la arquitectura base del modelo

In [ ]:
import tensorflow as tf
import os
from tensorflow.keras import Sequential, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

# Dynamically determine the number of classes from the training data generator
num_classes = len(train_generator.class_indices)

# Set random seed for reproducibility
tf.random.set_seed(42)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

### Compilación del modelo

In [ ]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
model.summary()

### Configuración de callbacks: Early stopping, un learning rate reducer y guardado del modelo para posterior uso

In [ ]:
# Callbacks for training
callbacks = [
    EarlyStopping(
        monitor='val_loss', 
        patience=10, 
        restore_best_weights=True, 
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        verbose=1,
        min_lr=1e-6
    ),
    ModelCheckpoint(
        filepath=os.path.join("model_checkpoints", "best_model.keras"), 
        save_best_only=True, 
        monitor='val_loss', 
        verbose=1
    )
]

### Entrenamiento del modelo personalizado

In [ ]:
import json

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=callbacks,
    class_weight=class_weights_dict
)

# Save training history
with open('training_history.json', 'w') as f:
    json.dump(history.history, f)

# Save the model in .keras format
model.save('my_model.keras')

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

## Visualización de los resultados

In [ ]:
import json
import matplotlib.pyplot as plt

# Load the training history
with open("training_history.json", "r") as f:
    history = json.load(f)  # This is already a dictionary

# Plot and save accuracy and loss
plt.figure(figsize=(12, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history['accuracy'], label='Train Accuracy')  # Access the dictionary directly
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')
plt.savefig("accuracy_plot.png")  # Save the accuracy plot

# Loss
plt.subplot(1, 2, 2)
plt.plot(history['loss'], label='Train Loss')  # Access the dictionary directly
plt.plot(history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.savefig("loss_plot.png")  # Save the loss plot

plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict on the test set
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class indices
y_true = test_generator.classes  # True class labels

# Classification report
report = classification_report(
    y_true, 
    y_pred_classes, 
    target_names=list(test_generator.class_indices.keys())
)
print("Classification Report:")
print(report)

# Save the classification report to a text file
with open("classification_report.txt", "w") as f:
    f.write("Classification Report\n")
    f.write(report)

# Confusion matrix
print("Confusion Matrix:")
cm = confusion_matrix(y_true, y_pred_classes)
print(cm)

import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=test_generator.class_indices.keys(), 
            yticklabels=test_generator.class_indices.keys())
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig("confusion_matrix.png")  # Save the confusion matrix plot
plt.show()